# ...

In [1]:
%matplotlib inline
%load_ext autoreload
# use %autoreload command to reload all libraries

In [2]:
from IPython.core.debugger import set_trace
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
#InteractiveShell.ast_node_interactivity = "last_expr"

In [3]:
# Python path fixing so we can import libraries
import sys
import os
sys_paths = ['../', # 
            ]
for p in sys_paths:
    p = os.path.abspath(p)
    if p not in sys.path:
        sys.path.append(p)

In [4]:
from hands.data import *
from hands.multiloss import *
from hands.hands_loss import *
from hands.notebook_utils import *
from hands.utils import * 

In [5]:
print(f"Pytorch: {torch.__version__}, Torchvision: {torchvision.__version__}, Fastai: {fastai.__version__}")

Pytorch: 1.0.0.dev20181009, Torchvision: 0.2.1, Fastai: 1.0.25.dev0


In [6]:
%autoreload

In [7]:
model_dir = "models/30_01"
data_path = pathlib.Path('/home/ohu/koodi/kesken/hands_old/custom_01/data')

# Setup

In [8]:
#img_org_dir = '/home/ohu/koodi/kesken/hands_data/data/jpg_exp/**/*fist/*.jpg'
img_org_dir = '/home/ohu/koodi/kesken/hands_data/data/train/**/*.jpg'
img_fnames_all = glob.glob(img_org_dir, recursive=True)
split = int(.9*len(img_fnames_all))
split = [split, len(img_fnames_all)-split]
img_fnames_arr = [list(sub) for sub in torch.utils.data.random_split(img_fnames_all, split)]

img_fnames_arr.append(glob.glob('/home/ohu/koodi/kesken/hands_data/data/test/**/*.jpg', recursive=True))

In [9]:
pickle.dump(img_fnames_arr, open(data_path/'splits/06.pk', "wb"))

In [10]:
_ = [len(a) for a in img_fnames_arr]
_
[a/sum(_) for a in _]

[13183, 1465, 4723]

[0.6805534045738475, 0.07562851685509267, 0.24381807857105983]

# Model / Learner Setup

In [11]:
from hands.models.model_darknet_customized import *

In [12]:
model = ModelDarknetCustomized(num_classes=len(HandItems.CLASS_NAMES),
                               darknet_layers=[1,2,8,8,4], darknet_output=1024)

In [13]:
_ = model.cuda().train()
learn = fastai.Learner(data=get_default_databunch(img_fnames_arr, size=320),
                       model=model, loss_func=HandsLoss())
learn.metrics = [HandsAccuracy()]
learn.callback_fns.append(HandleAvgMultiLoss)
learn.path = pathlib.Path('../data/')
learn.split((model.pre_0[0][0], ))
learn.model_dir = model_dir

In [14]:
if not os.path.isdir(learn.path/learn.model_dir):
    os.mkdir(learn.path/learn.model_dir)

In [15]:
_ = model_load_backbone(model, '../data/models/yolov3_coco_01.h5')

Loaded to: backbone.base.conv.weight -> backbone.darknet_4.4.conv2.bn.running_var.
Skipped from: yolo_0_pre.branch.0.conv.weight...


In [16]:
learn.freeze()
model_requires_grad_info(model)

backbone.base.conv.weight: False
...
pre_0.14_convbatch.0.weight: True
...
pre_0.15_conv.bias: True


In [45]:
import gc
def empty_cuda_cache():
    for _ in range(2):
        gc.collect()
        torch.cuda.empty_cache()
        time.sleep(1)

In [46]:
empty_cuda_cache()

# Learner & Fitting

In [17]:
sz = 13*32
sz
learn.data = get_default_databunch(img_fnames_arr, size=sz, bs=16)

416

In [18]:
learn.fit_one_cycle(3, max_lr=1e-2)

Total time: 18:39
epoch  train_loss  valid_loss  accuracy  loss_conf  loss_cls  loss_coord  loss_angle 
1      15.445598   19.191763   0.420478  1.158999   15.209193  1.208138    1.615433    (06:26)
2      9.224243    9.322507    0.564505  0.956627   6.488301  0.854443    1.023136     (06:07)
3      5.368410    5.797952    0.692150  0.881636   3.631513  0.673787    0.611016     (06:05)



In [19]:
learn.save('1')

In [20]:
learn.validate(learn.data.test_dl)

[25.154396, 0.5028583527419014]

In [21]:
corr, wrongs, w_samples, w_idxs = calculate_accuracy_files(model, learn.data.test_ds.fnames, learn.data.test_dl.size, max_samples=0)
corr, wrongs

(0.46855811983908535,
 {'not_enough_preds': 671,
  'too_many_preds': 18,
  'wrong_class': 1030,
  'too_far': 4,
  'wrong_direction': 787})

In [22]:
learn.unfreeze()

In [23]:
model_requires_grad_info(model)

backbone.base.conv.weight: True
...
pre_0.15_conv.bias: True


In [24]:
learn.fit_one_cycle(30, max_lr=1e-4)

Total time: 3:11:49
epoch  train_loss  valid_loss  accuracy  loss_conf  loss_cls  loss_coord  loss_angle
1      5.240332    5.799206    0.703754  0.877394   3.624055  0.689077    0.608680    (06:23)
2      4.771288    4.715448    0.701024  0.832846   2.644024  0.652720    0.585858    (06:21)
3      4.428966    4.955088    0.722184  0.823944   2.958671  0.626628    0.545844    (06:20)
4      4.162064    4.161051    0.720137  0.784161   2.321392  0.559931    0.495567    (06:22)
5      4.145215    3.947315    0.753584  0.791345   2.114428  0.558231    0.483310    (06:22)
6      4.467503    3.955883    0.738567  0.748717   2.229196  0.541669    0.436302    (06:21)
7      3.806620    4.437783    0.721502  0.718687   2.822373  0.513639    0.383084    (06:21)
8      3.235505    4.053235    0.744027  0.725803   2.414838  0.498736    0.413859    (06:25)
9      3.502134    3.643628    0.749488  0.710138   1.977045  0.496456    0.459990    (06:22)
10     2.926832    3.046877    0.742662  0.669922

In [25]:
learn.save('2')

In [26]:
learn.validate(learn.data.test_dl)

[21.029224, 0.6764768155833157]

In [27]:
corr, wrongs, w_samples, w_idxs = calculate_accuracy_files(model, learn.data.test_ds.fnames, learn.data.test_dl.size, max_samples=0)
corr, wrongs

(0.6612322676265086,
 {'not_enough_preds': 439,
  'too_many_preds': 40,
  'wrong_class': 536,
  'too_far': 1,
  'wrong_direction': 584})

### Size 512

In [37]:
sz = 16*32
sz
learn.data = get_default_databunch(img_fnames_arr, size=sz, bs=16)

512

In [40]:
corr, wrongs, w_samples, w_idxs = calculate_accuracy_files(model, learn.data.test_ds.fnames, learn.data.test_dl.size, max_samples=0)
corr, wrongs

(0.6826169807325853,
 {'not_enough_preds': 450,
  'too_many_preds': 25,
  'wrong_class': 494,
  'too_far': 0,
  'wrong_direction': 530})

In [29]:
learn.fit_one_cycle(30, max_lr=1e-4)

Total time: 4:05:00
epoch  train_loss  valid_loss  accuracy  loss_conf  loss_cls  loss_coord  loss_angle
1      0.828614    1.043190    0.897611  0.486077   0.160641  0.268804    0.127667    (08:08)
2      0.878656    1.037090    0.894198  0.491858   0.154332  0.260618    0.130283    (08:09)
3      0.874033    1.032966    0.891468  0.505227   0.117308  0.274074    0.136357    (08:08)
4      0.922832    1.132341    0.890102  0.502927   0.193303  0.300298    0.135812    (08:09)
5      1.124886    1.219229    0.896928  0.504176   0.287434  0.285627    0.141992    (08:09)
6      1.308714    1.154843    0.901024  0.530169   0.180139  0.282341    0.162194    (08:08)
7      1.180694    1.315985    0.882594  0.516843   0.323556  0.317432    0.158154    (08:09)
8      1.582386    1.428283    0.865529  0.546873   0.355256  0.330186    0.195969    (08:08)
9      1.279384    1.448221    0.869625  0.516251   0.427137  0.313831    0.191002    (08:09)
10     1.589956    2.333200    0.845051  0.547034

In [30]:
learn.save('3_512')

In [31]:
learn.validate(learn.data.test_dl)

[17.784304, 0.7228456489519374]

In [32]:
corr, wrongs, w_samples, w_idxs = calculate_accuracy_files(model, learn.data.test_ds.fnames, learn.data.test_dl.size, max_samples=0)
corr, wrongs

(0.7124708871479991,
 {'not_enough_preds': 462,
  'too_many_preds': 8,
  'wrong_class': 468,
  'too_far': 1,
  'wrong_direction': 419})

### Size 608

In [41]:
_ = learn.load('3_512')

In [42]:
sz = 19*32
sz
learn.data = get_default_databunch(img_fnames_arr, size=sz, bs=8)

608

In [43]:
learn.validate(learn.data.test_dl)

[8.397524, 0.733432140588609]

In [44]:
corr, wrongs, w_samples, w_idxs = calculate_accuracy_files(model, learn.data.test_ds.fnames, learn.data.test_dl.size, max_samples=0)
corr, wrongs

(0.7188227821300022,
 {'not_enough_preds': 489,
  'too_many_preds': 10,
  'wrong_class': 385,
  'too_far': 0,
  'wrong_direction': 444})

In [47]:
corr, wrongs, w_samples, w_idxs = calculate_accuracy_files(model, learn.data.test_ds.fnames, learn.data.test_dl.size,
                                                           get_results_args={'conf_thresh': .20})
corr, wrongs

(0.7484649587126826,
 {'not_enough_preds': 201,
  'too_many_preds': 56,
  'wrong_class': 453,
  'too_far': 0,
  'wrong_direction': 478})

In [34]:
learn.fit_one_cycle(30, max_lr=1e-5)

epoch,train_loss,valid_loss,accuracy,loss_conf,loss_cls,loss_coord,loss_angle


RuntimeError: CUDA error: out of memory

In [ ]:
learn.save('4_608')

In [ ]:
learn.validate(learn.data.test_dl)

In [ ]:
corr, wrongs, w_samples, w_idxs = calculate_accuracy_files(model, learn.data.test_ds.fnames, learn.data.test_dl.size, max_samples=0)
corr, wrongs

In [ ]:
learn.fit_one_cycle(30, max_lr=1e-5)

In [ ]:
learn.save('5_608')

In [ ]:
learn.validate(learn.data.test_dl)

In [ ]:
corr, wrongs, w_samples, w_idxs = calculate_accuracy_files(model, learn.data.test_ds.fnames, learn.data.test_dl.size, max_samples=0)
corr, wrongs

In [ ]:
learn.fit_one_cycle(30, max_lr=1e-5)

In [ ]:
learn.save('6_608')

In [ ]:
learn.validate(learn.data.test_dl)

In [ ]:
corr, wrongs, w_samples, w_idxs = calculate_accuracy_files(model, learn.data.test_ds.fnames, learn.data.test_dl.size, max_samples=0)
corr, wrongs